
# 06 — Practice Notes: Model Risk & Finance Use-Cases (Copulas)

This notebook analyzes two chapters:

- **01_model_risk_pitfalls.md** — model risk sources, diagnostics, and mitigations.
- **02_finance_use_cases.md** — portfolio aggregation, credit, market dependence, stress testing.


In [1]:

import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

FIG_DIR = Path("../docs/assets/figures/06_practice_notes")
FIG_DIR.mkdir(parents=True, exist_ok=True)

def savefig(path):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    plt.tight_layout()
    plt.savefig(path, format="svg", bbox_inches="tight")
    plt.close()

rng = np.random.default_rng(42)

def sample_gaussian_copula(n=10000, rho=0.6):
    import numpy as np
    from scipy.stats import norm
    z = rng.standard_normal((n,2))
    L = np.array([[1.0, 0.0],[rho, np.sqrt(1-rho**2)]])
    x = z @ L.T
    u = norm.cdf(x)
    return u

def sample_t_copula(n=10000, rho=0.6, nu=4):
    import numpy as np
    from scipy.stats import t
    z = rng.standard_normal((n,2))
    L = np.array([[1.0, 0.0],[rho, np.sqrt(1-rho**2)]])
    x = z @ L.T
    g = rng.chisquare(nu, size=n) / nu
    x = x / np.sqrt(g)[:,None]
    u = t.cdf(x, df=nu)
    return u



## A. Figures for *02_finance_use_cases.md*


In [2]:

# Figure: finance_risk_pipeline.svg
fig, ax = plt.subplots(figsize=(8,3))
ax.axis("off")
boxes = [
    ("Data & Marginals\n(F_i)", (0.05, 0.5)),
    ("Calibration\n(C_θ | U)", (0.28, 0.5)),
    ("Simulation\n(U~C_θ)", (0.51, 0.5)),
    ("Aggregation\n(P&L)", (0.74, 0.5)),
    ("Validation\n(PIT/GoF/Backtest)", (0.9, 0.5)),
]
for label, (x,y) in boxes:
    ax.add_patch(plt.Rectangle((x-0.09, y-0.12), 0.18, 0.24, fill=False))
    ax.text(x, y, label, ha="center", va="center")
for i in range(len(boxes)-1):
    x1, y1 = boxes[i][1]
    x2, y2 = boxes[i+1][1]
    ax.annotate("", xy=(x2-0.10, y2), xytext=(x1+0.10, y1),
                arrowprops=dict(arrowstyle="->", lw=1.5))
savefig(FIG_DIR/"finance_risk_pipeline.svg")


In [3]:

# Figure: finance_credit_tail.svg
N = 1000; PD = 0.01; LGD = 1.0; scenarios = 2000
rho = 0.2; nu = 4
from scipy.stats import norm, t as tdist
losses_g, losses_t = [], []
for _ in range(scenarios):
    Y = rng.standard_normal()
    eps = rng.standard_normal(N)
    Zi_g = np.sqrt(rho)*Y + np.sqrt(1-rho)*eps
    thr = norm.ppf(PD)
    d_g = (Zi_g <= thr).sum()
    losses_g.append(d_g*LGD)

    Yt = tdist.rvs(df=nu, random_state=rng)
    epst = tdist.rvs(df=nu, size=N, random_state=rng)
    Zi_t = np.sqrt(rho)*Yt + np.sqrt(1-rho)*epst
    thr_t = tdist.ppf(PD, df=nu)
    d_t = (Zi_t <= thr_t).sum()
    losses_t.append(d_t*LGD)

q = 0.99
var_g = np.quantile(losses_g, q)
var_t = np.quantile(losses_t, q)

fig, ax = plt.subplots(figsize=(6,4))
bins = np.arange(0, max(max(losses_g), max(losses_t))+5, 5)
ax.hist(losses_g, bins=bins, density=True, alpha=0.4, label=f"Gaussian (VaR@99%≈{var_g:.0f})")
ax.hist(losses_t, bins=bins, density=True, alpha=0.4, label=f"t, ν={nu} (VaR@99%≈{var_t:.0f})")
ax.set_xlabel("Portfolio losses (counts of defaults)")
ax.set_ylabel("Density")
ax.set_title("Credit portfolio tail simulation")
ax.legend()
savefig(FIG_DIR/"finance_credit_tail.svg")


In [4]:

# Figure: finance_dependence_heatmap.svg
labels = ["Equity","Credit","Rates","FX","Commod","Vol"]
tau = np.array([
    [1.0,  0.45, -0.10,  0.05,  0.10, -0.35],
    [0.45, 1.0,  -0.20,  0.00,  0.15, -0.40],
    [-0.10,-0.20, 1.0,  -0.05, -0.15,  0.20],
    [0.05, 0.00, -0.05,  1.0,   0.05, -0.10],
    [0.10, 0.15, -0.15,  0.05,  1.0,  -0.25],
    [-0.35,-0.40, 0.20, -0.10, -0.25,  1.0],
])
fig, ax = plt.subplots(figsize=(5.5,5))
im = ax.imshow(tau, origin="upper", vmin=-1, vmax=1)
ax.set_xticks(range(len(labels))); ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_yticks(range(len(labels))); ax.set_yticklabels(labels)
ax.set_title("Kendall τ across asset classes")
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("τ")
savefig(FIG_DIR/"finance_dependence_heatmap.svg")



## B. Figures for *01_model_risk_pitfalls.md*


In [5]:

# Figure: model_risk_layers.svg
fig, ax = plt.subplots(figsize=(6.5,4))
ax.axis("off")
layers = [
    ("Marginals", (0.1, 0.75)),
    ("Copula\n(family & params)", (0.1, 0.5)),
    ("Implementation\n(numerics, seeds)", (0.1, 0.25)),
]
for label, (x,y) in layers:
    ax.add_patch(plt.Rectangle((x, y-0.1), 0.8, 0.18, fill=False))
    ax.text(x+0.02, y, label, va="center")
ax.annotate("", xy=(0.9, 0.33), xytext=(0.9, 0.67),
            arrowprops=dict(arrowstyle="<->", lw=1.5))
ax.text(0.92, 0.5, "Risk\nPropagation", va="center")
savefig(FIG_DIR/"model_risk_layers.svg")


In [6]:

# Figure: model_risk_tail_bias.svg
rho_true = 0.6; nu_true = 4; n = 10000
U = sample_t_copula(n=n, rho=rho_true, nu=nu_true)
alpha = 0.95
emp_tail = np.mean((U[:,0] > alpha) & (U[:,1] > alpha))
from scipy.stats import kendalltau
tau_hat, _ = kendalltau(U[:,0], U[:,1])
rho_g = np.sin(np.pi * tau_hat / 2)
Ug = sample_gaussian_copula(n=n, rho=rho_g)
gauss_tail = np.mean((Ug[:,0] > alpha) & (Ug[:,1] > alpha))

fig, ax = plt.subplots(figsize=(6,5))
ax.scatter(U[::10,0], U[::10,1], s=4, alpha=0.4, label="Empirical (t-copula)")
ax.axvline(alpha, linestyle="--"); ax.axhline(alpha, linestyle="--")
ax.text(0.02, 0.98, f"Empirical τ≈{tau_hat:.2f}\nTail@0.95 Emp≈{emp_tail:.3f}\nGauss≈{gauss_tail:.3f}",
        transform=ax.transAxes, va="top")
ax.set_xlabel("U1"); ax.set_ylabel("U2")
ax.set_title("Tail underestimation by Gaussian vs heavy-tailed dependence")
savefig(FIG_DIR/"model_risk_tail_bias.svg")
